In [74]:
#HW4 functions to use in HW5

#local stiffness matrix
import numpy as np
import math as m

#initialize k_local
def define_ke_local(E, A, L):
    k = np.zeros((4,4))
    
    k[0][0] = (E*A/L)
    k[2][2] = (E*A/L)
    k[0][2] = (-E*A/L)
    k[2][0] = (-E*A/L)
    
    return k

def define_rotation(theta):
    unit_rot = [[np.cos(theta), np.sin(theta)], [-np.sin(theta), np.cos(theta)]]
    beta = np.zeros((4,4))
    beta[0:2,0:2] = unit_rot
    beta[2:4,2:4] = unit_rot
    return beta

def calc_length(i_coords, j_coords):
    return np.linalg.norm((j_coords - i_coords))

def calc_orientation(i_coords, j_coords):
    delta = j_coords - i_coords
    return np.arctan2(delta[1], delta[0])

def calc_dof(i_node_num, j_node_num):
    dof_i = [i_node_num * 2, i_node_num * 2 + 1]
    dof_j = [j_node_num * 2, j_node_num * 2 + 1]
    dof = dof_i + dof_j
    return dof

def define_ke_global(ke_local, beta):
    ke_global = np.dot((np.dot(np.transpose(beta), ke_local)), beta)
    return ke_global


## DISCLAIMER: I had difficulties setting up a loop using python for this problem.
## I was able to set it up using the specific truss case, however.
def assemble_big_mac(num_dof, ke_global, global_dof):
    big_mac = np.zeros((num_dof, num_dof))
    for i in range(len(ke_global)):
        if i == 0 or i == 2:
            big_mac[i:i+4,i:i+4] += ke_global[i]
        elif i == 1:
            big_mac[0:2,0:2] += ke_global[i][0:2,0:2]
            big_mac[0:2,4:6] += ke_global[i][0:2,2:4]
            big_mac[4:6,0:2] += ke_global[i][2:4,0:2]
            big_mac[4:6,4:6] += ke_global[i][2:4,2:4]
    return big_mac



In [77]:


nodes = np.array([[0.0, 0.0], [-100.0,-100.0], [100.0, -100.0]])
mat_props = [29000000.0, 10000.0, 15000000.0]
section_props = [1.0, 2.0, 3.6]
elements = [[0, 1, 0, 1], [0, 2, 2, 0], [1, 2, 1, 2]]
s = [2,3,4,5]
F = np.zeros(np.size(nodes))
F[0] =  100000.0
u = np.zeros(np.size(nodes))
#print(len(elements))
A = [section_props[elements[i][3]] for i in range(len(elements))]
#print(A)
E = [mat_props[elements[i][2]] for i in range(len(elements))]
#print(E)
L = [calc_length(nodes[elements[i][0]], nodes[elements[i][1]]) for i in range(len(elements))]
#print(L)
theta = [calc_orientation(nodes[elements[i][0]], nodes[elements[i][1]]) for i in range(len(elements))]
#print(theta)
ke_local = [define_ke_local(E[i], A[i], L[i]) for i in range(len(L))]
#print(ke_local)
beta = [define_rotation(theta[i]) for i in range(len(theta))]
#print(beta)
ke_global = [define_ke_global(ke_local[i], beta[i]) for i in range(len(ke_local))]
#print(ke_global)
element_global_dof = [calc_dof(elements[i][0], elements[i][1]) for i in range(len(elements))]
#print(element_global_dof)
num_dof = np.shape(nodes)[0] * 2
K = assemble_big_mac(num_dof, ke_global, element_global_dof)

temp_p = []
for i in range(num_dof):
    temp_p.append(i)
p = np.setdiff1d(temp_p, s)
Kpp = K[p[0]:p[-1] + 1, p[0]:p[-1] + 1]
Kss = K[s[0]:s[-1] + 1, s[0]:s[-1] + 1]
Kps = K[p[0]:p[-1] + 1, s[0]:s[-1] + 1]
Ksp = K[s[0]:s[-1] + 1, p[0]:p[-1] + 1]
Fp = F[p]
us = u[s]
up = np.dot(np.linalg.inv(Kpp),(Fp - np.dot(Kps, us)))
u[p] = up
Fs = np.dot(Ksp, up) + np.dot(Kss, us)
u_element_global = [u[element_global_dof[i]] for i in range(len(element_global_dof))]
displacement = [np.dot(beta[i], u_element_global[i]) for i in range(len(u_element_global))]
P = [np.dot(ke_local[i], displacement[i]) for i in range(len(ke_local))] 
print("Displacement Field:\n")
print(displacement)
print()
print("Reactions:\n")
print(Fs)
print()
print("Internal Forces:\n")
print(P)

Displacement Field:

[array([-0.17241379,  0.66666667,  0.        ,  0.        ]), array([0.66666667, 0.17241379, 0.        , 0.        ]), array([0., 0., 0., 0.])]

Reactions:

[-50000. -50000. -50000.  50000.]

Internal Forces:

[array([-70710.67811865,      0.        ,  70710.67811865,      0.        ]), array([ 70710.67811865,      0.        , -70710.67811865,      0.        ]), array([0., 0., 0., 0.])]
